# Implementation of LSTM to Motion Classification Application
Implementation of LSTM technique to motion detection application. Developed by James Youngchae Chee.

### 1. Setting Up Environment

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 2. Importing Data
This dataset contains fall, swing and still gyro and accelerometer data.

In [14]:
train_data = [pd.read_csv('./data/fall.csv',header=None),
             pd.read_csv('./data/swing.csv',header=None),
             pd.read_csv('./data/still.csv',header=None)]
test_data = [pd.read_csv('./data/falltest.csv',header=None),
             pd.read_csv('./data/swingtest.csv',header=None),
             pd.read_csv('./data/stilltest.csv',header=None)]

### 3. Examining Data
Each Data has <b>1200</b> dimensions --> Stacked numerous sensor data together
<br>Each Retrieval of sesor data looks like (linear_accel x,y,z, Gyro x,y,z)
<br><b>Number of train_data : 50
<br><b>Number of test_data : 10

In [24]:
train_data[0].shape

(50, 1200)

In [25]:
test_data[0].shape

(10, 1200)